In [1]:
import requests
import os
import json
from dotenv import load_dotenv
import pathlib
import pandas as pd

# name the file we want
env_file_name = 'lrhf97_twit_cred.env'

# designate the path of the file
curr_file = pathlib.Path.cwd()

# join them
env_path = curr_file.joinpath(env_file_name)

# load the file
load_dotenv(env_path)

bearer_token = os.getenv("BEARER_TOKEN")


def create_url():
    # Replace with user ID below - RidgeSupply =3413455348
    user_id = 3413455348
    # Right now the max users is hard coded as 10
    max_users = 10
    
    return "https://api.twitter.com/2/users/{}/followers?max_results={}".format(user_id,max_users)

def get_params():
    return {'user.fields': 'id,name,username,created_at,description,public_metrics,verified,location'}

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FollowersLookupPython"
    return r


def connect_to_endpoint(url, params):
    # params['next_token'] = next_token # param object recieved form the create_url function
    response = requests.request("GET", url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main():
    url = create_url()
    params = get_params()
    json_response = connect_to_endpoint(url, params)
    # print(json.dumps(json_response, indent=4, sort_keys=True))
    return json_response

ridge_followers= main()

200


In [6]:
ridge_followers['data'][0]

{'name': 'Andrew Morrison',
 'location': 'Charlotte, USA',
 'username': 'AWalkerMorrison',
 'created_at': '2022-02-03T18:02:43.000Z',
 'verified': False,
 'id': '1489298184534908930',
 'description': 'Husband, father, cyclist, student @NCSUgeospatial.',
 'public_metrics': {'followers_count': 29,
  'following_count': 1378,
  'tweet_count': 30,
  'listed_count': 0}}

In [3]:
df=pd.DataFrame.from_dict(ridge_followers['data'])
df.head(5)

,name,location,username,created_at,verified,id,description,public_metrics
0,Andrew Morrison,"Charlotte, USA",AWalkerMorrison,2022-02-03T18:02:43.000Z,False,1489298184534908930,"Husband, father, cyclist, student @NCSUgeospat...","{'followers_count': 29, 'following_count': 137..."
1,Kyle,Texas,Pedalpusher2941,2022-05-10T01:35:39.000Z,False,1523839034723872776,Just a guy nothin fancy - A little bit of this...,"{'followers_count': 20, 'following_count': 233..."
2,Daniel Williams,"Kentucky, USA",wykydgt,2009-06-23T14:15:03.000Z,False,49985899,"Dad, husband, paramedic, adaptive cyclist.","{'followers_count': 212, 'following_count': 56..."
3,Nathan Carroll,"Pennsylvania, USA",_Nathanael_C,2016-12-06T15:08:04.000Z,False,806153237107720192,|Artist|Art Educator|Husband|Father|Cyclist|Hi...,"{'followers_count': 54, 'following_count': 81,..."
4,Anna Ladas,NaN,bike_bliss,2015-08-05T15:40:03.000Z,False,3307108069,,"{'followers_count': 1, 'following_count': 27, ..."


In [4]:
df.description[1]

'Just a guy nothin fancy - A little bit of this and a little bit of that. Ride bikes- The Office - Sports-  Over thinker w/ a touch of western.'

## Find likes

In [7]:

def create_liked_url():
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    tweet_fields = "tweet.fields=text,author_id"
    user_fields = "user.fields=username"
    # Be sure to replace your-user-id with your own user ID or one of an authenticating user
    # You can find a user ID by using the user lookup endpoint
    id = 3413455348
    # You can adjust ids to include a single Tweets.
    # Or you can add to up to 100 comma-separated IDs
    liked_url = "https://api.twitter.com/2/users/{}/liked_tweets".format(id)
    return liked_url, tweet_fields, user_fields

def connect_to_liked_endpoint(liked_url, tweet_fields):
    response = requests.request(
        "GET", liked_url, auth=bearer_oauth, params=tweet_fields)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

def main():
    liked_url, tweet_fields, user_fields = create_liked_url()
    json_response = connect_to_liked_endpoint(liked_url, tweet_fields)
    return json_response

liked_texts =main()
liked_texts['data'][0]

200


{'author_id': '16949790',
 'id': '1506734904549249027',
 'text': '@sdbeachgirl9 @philgaimon @ridgesupply'}

## Retweets

In [8]:

def create_url():
    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
    user_fields = "user.fields=created_at,description"
    # You can replace the ID given with the Tweet ID you wish to lookup Retweeting users for
    # You can find an ID by using the Tweet lookup endpoint
    id = "1437987598790582277"
    # You can adjust ids to include a single Tweets.
    # Or you can add to up to 100 comma-separated IDs
    url = "https://api.twitter.com/2/tweets/{}/retweeted_by".format(id)
    return url, user_fields


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RetweetedByPython"
    return r

def connect_to_endpoint(url, user_fields):
    response = requests.request("GET", url, auth=bearer_oauth, params=user_fields)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main():
    url, user_fields = create_url()
    json_response = connect_to_endpoint(url, user_fields)
    return json_response

retweets = main()
retweets['data'][0]



200


{'created_at': '2012-03-29T19:34:36.000Z',
 'description': "Active Optimist. Humanist Principles: https://t.co/Kbx0pLKDFV… Best ideas on Europe's future: https://t.co/qL7Wi1SSl8 UK: https://t.co/ZmulUQVqCP",
 'name': '🐟 🇪🇺🇺🇦Allan Paterson, @RebootGB #FBResist',
 'id': '540223123',
 'username': 'AllanGPaterson'}

In [9]:
df_retweet=pd.DataFrame. from_dict(retweets['data'])
df_retweet.head(5)


,created_at,description,name,id,username
0,2012-03-29T19:34:36.000Z,Active Optimist. Humanist Principles: https://...,"🐟 🇪🇺🇺🇦Allan Paterson, @RebootGB #FBResist",540223123,AllanGPaterson
1,2018-06-17T22:15:33.000Z,No DM's \nSeeker of truth and justice.\n#Resis...,Melissa Johnson,1008473248063991808,eattherich146
2,2009-08-12T04:39:34.000Z,,Gregg Schwinn,64939212,605erLBM
3,2016-10-15T04:03:33.000Z,"PhD, UC Berkeley: School Psychologist \n#Warri...",Lady Cybele,787141837286875137,lady_cybele
4,2019-04-09T16:10:36.000Z,,Mctrish,1115648212231016450,Mctrish5
